
# Please provide the answers & outputs for the below 10 questions
### (please upload the submissions in notebook html format: File -> Download as .html page)                                  
### Dataset: diabetes.csv

#### 1. Perform the EDA - Identify the total Numeric and categorical variables,  Null values for dataset , Cardinality of all variables 
#### 2. Design a udf function for BloodPressure variable to create an indicator dervied variable -condition:  blood pressure >70 as 1 else 0 end 
#### 3. Run the summary statistics for dataframe
#### 4. Create a new column - skinthickness_group new variable using SkinThickness variable 
1. condition cutoff: SkinThickness in 0-15 as  SK1
2. SkinThickness in >15 & <20  as sk2
3. SkinThickness in =20 and in=45 as sk3
4. SkinThickness in greater than 45 as sk4
5. remaining ones in else group other 
## Implement the below functions  for  few variables (2 at least)

#### 5. String Indexer
#### 6. One hot encoder
#### 7. Label encoder
#### 8. standard scaler
#### 9. Vector assembler

#### 10. Convert df to pandas and Perform the pearson coefficient  - Target variable (Outcome) vs all input variables individually

In [1]:
import os
 
print("username:" , os.getlogin())
from datetime import date

today = date.today()
print("Today's date:", today)

username: Usuario
Today's date: 2023-11-10


In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import * 
import pyspark.sql.functions as F
from pyspark.sql.functions import col, asc,desc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pyspark.sql import SQLContext
from pyspark.mllib.stat import Statistics
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler,StandardScaler
from pyspark.ml import Pipeline
from sklearn.metrics import confusion_matrix

spark=SparkSession.builder \
.master ("local[*]")\
.appName("part3")\
.getOrCreate()

In [4]:
sc=spark.sparkContext
sqlContext=SQLContext(sc)

C:\Users\Usuario\Downloads\spark-3.5.0-bin-hadoop3\python\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
import os
os.getcwd()

'C:\\Users\\Usuario\\Documents\\DataScience\\BAN5753\\Week12\\Exercise'

In [6]:
from platform import python_version

print(python_version())

3.9.12


In [7]:
sc.version #spark version

'3.5.0'

## Read File

In [8]:
df=spark.read \
 .option("header","True")\
 .option("inferSchema","True")\
 .option("sep",",")\
 .csv("diabetes.csv")
print("There are",df.count(),"rows",len(df.columns),
      "columns" ,"in the data.") 

There are 768 rows 9 columns in the data.


## Show Sample Data

In [9]:
df.show(4)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 4 rows



## Data Types of Columns

In [10]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



### 1. Perform the EDA - Identify the total Numeric and categorical variables,  Null values for dataset , Cardinality of all variables 

In [11]:
String_columnList = [item[0] for item in df.dtypes if item[1].startswith('string')]
Int_columnList = [item[0] for item in df.dtypes if item[1].startswith('integer')]
Double_columnList = [item[0] for item in df.dtypes if item[1].startswith('double')]

Int_columnList, Double_columnList, String_columnList
Numerical = Int_columnList+Double_columnList

In [12]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas()

,summary,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,Age,Outcome
0,count,768,768,768,768,768,768,768
1,mean,3.8450520833333335,120.89453125,69.10546875,20.536458333333332,79.79947916666667,33.240885416666664,0.3489583333333333
2,stddev,3.36957806269887,31.97261819513622,19.355807170644777,15.952217567727642,115.24400235133803,11.760231540678689,0.476951377242799
3,min,0,0,0,0,0,21,0
4,max,17,199,122,99,846,81,1


In [13]:
numeric_features2 = [t[0] for t in df.dtypes if t[1] == 'double']
df.select(numeric_features2).describe().toPandas()

,summary,BMI,DiabetesPedigreeFunction
0,count,768,768
1,mean,31.992578124999977,0.4718763020833327
2,stddev,7.884160320375441,0.331328595012775
3,min,0.0,0.078
4,max,67.1,2.42


In [14]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).toPandas().head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,0,0,0,0,0,0,0,0


In [15]:
Dict_Null = {col:df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null

{'Pregnancies': 0,
 'Glucose': 0,
 'BloodPressure': 0,
 'SkinThickness': 0,
 'Insulin': 0,
 'BMI': 0,
 'DiabetesPedigreeFunction': 0,
 'Age': 0,
 'Outcome': 0}

#### Looking at the results, there are 8 numerical variables and 1 categorical variable. The categorical variable is "outcome". There doesn't seem to be any null values for the dataset. Every variable has 768 records and we have a total of 9 variables. 

### 2. Design a udf function for BloodPressure variable to create an indicator dervied variable -condition: blood pressure >70 as 1 else 0 end

In [16]:
from pyspark.sql.functions import udf
y_udf = udf(lambda y: 1 if y>70 else 0, IntegerType())

df=df.withColumn("BP>70", y_udf('BloodPressure'))

In [17]:
df.show(6)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+-----+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|BP>70|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+-----+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|    1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|    0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|    0|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|    0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|    0|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|    1|
+-----------+-------+-------------+-----------

### 3. Run the summary statistics for dataframe

In [18]:
numeric_features2 = [t[0] for t in df.dtypes if t[1] == 'double']
df.select(numeric_features2).describe().toPandas()

,summary,BMI,DiabetesPedigreeFunction
0,count,768,768
1,mean,31.992578124999977,0.4718763020833327
2,stddev,7.884160320375441,0.331328595012775
3,min,0.0,0.078
4,max,67.1,2.42


In [19]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas()

,summary,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,Age,Outcome,BP>70
0,count,768,768,768,768,768,768,768,768
1,mean,3.8450520833333335,120.89453125,69.10546875,20.536458333333332,79.79947916666667,33.240885416666664,0.3489583333333333,0.51171875
2,stddev,3.36957806269887,31.97261819513622,19.355807170644777,15.952217567727642,115.24400235133803,11.760231540678689,0.476951377242799,0.5001884015974067
3,min,0,0,0,0,0,21,0,0
4,max,17,199,122,99,846,81,1,1


### 4. Create a new column - skinthickness_group new variable using SkinThickness variable 
1. condition cutoff: SkinThickness in 0-15 as  SK1
2. SkinThickness in >15 & <20  as sk2
3. SkinThickness in =20 and in=45 as sk3
4. SkinThickness in greater than 45 as sk4
5. remaining ones in else group other 

In [20]:
def udf_multiple(SkinThickness):
      if (SkinThickness >= 0 and SkinThickness <= 15):
        return 'SK1'
      elif (SkinThickness > 15 and SkinThickness < 20):
        return 'SK2'
      elif (SkinThickness == 20 and SkinThickness == 45):
        return 'SK3'
      elif (SkinThickness > 45):
        return 'SK4'
      else: return 'other'

education_udf = udf(udf_multiple)
df=df.withColumn("skinthickness_group", education_udf('SkinThickness'))

In [21]:
df.toPandas()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BP>70,skinthickness_group
0,6,148,72,35,0,33.6,0.627,50,1,1,other
1,1,85,66,29,0,26.6,0.351,31,0,0,other
2,8,183,64,0,0,23.3,0.672,32,1,0,SK1
3,1,89,66,23,94,28.1,0.167,21,0,0,other
4,0,137,40,35,168,43.1,2.288,33,1,0,other
...,...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0,1,SK4
764,2,122,70,27,0,36.8,0.340,27,0,0,other
765,5,121,72,23,112,26.2,0.245,30,0,1,other
766,1,126,60,0,0,30.1,0.349,47,1,0,SK1


## Implement the below functions  for  few variables (2 at least)

### 5. String Indexer

In [22]:
def udf_multiple(Outcome):
      if (Outcome == 0):
        return 'Zero'
      else: return 'One'

education_udf = udf(udf_multiple)
df=df.withColumn("Outcome_label", education_udf('Outcome'))

In [23]:
df.toPandas()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BP>70,skinthickness_group,Outcome_label
0,6,148,72,35,0,33.6,0.627,50,1,1,other,One
1,1,85,66,29,0,26.6,0.351,31,0,0,other,Zero
2,8,183,64,0,0,23.3,0.672,32,1,0,SK1,One
3,1,89,66,23,94,28.1,0.167,21,0,0,other,Zero
4,0,137,40,35,168,43.1,2.288,33,1,0,other,One
...,...,...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0,1,SK4,Zero
764,2,122,70,27,0,36.8,0.340,27,0,0,other,Zero
765,5,121,72,23,112,26.2,0.245,30,0,1,other,Zero
766,1,126,60,0,0,30.1,0.349,47,1,0,SK1,One


In [24]:
df2=df
df3=df

In [25]:
stringIndexer = StringIndexer()\
                 .setInputCol ("skinthickness_group")\
                 .setOutputCol ("skinthickness_groupIndex")

skinthickness_groupIndex_model=stringIndexer.fit(df2)
skinthickness_groupIndex_df=skinthickness_groupIndex_model.transform(df2)
skinthickness_groupIndex_df.toPandas().head(20)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BP>70,skinthickness_group,Outcome_label,skinthickness_groupIndex
0,6,148,72,35,0,33.6,0.627,50,1,1,other,One,0.0
1,1,85,66,29,0,26.6,0.351,31,0,0,other,Zero,0.0
2,8,183,64,0,0,23.3,0.672,32,1,0,SK1,One,1.0
3,1,89,66,23,94,28.1,0.167,21,0,0,other,Zero,0.0
4,0,137,40,35,168,43.1,2.288,33,1,0,other,One,0.0
5,5,116,74,0,0,25.6,0.201,30,0,1,SK1,Zero,1.0
6,3,78,50,32,88,31.0,0.248,26,1,0,other,One,0.0
7,10,115,0,0,0,35.3,0.134,29,0,0,SK1,Zero,1.0
8,2,197,70,45,543,30.5,0.158,53,1,0,other,One,0.0
9,8,125,96,0,0,0.0,0.232,54,1,1,SK1,One,1.0


In [26]:
stringIndexer = StringIndexer()\
                 .setInputCol ("Outcome_label")\
                 .setOutputCol ("Outcome_groupIndex")

Outcome_groupIndex_model=stringIndexer.fit(df2)
Outcome_groupIndex_df=Outcome_groupIndex_model.transform(df2)
Outcome_groupIndex_df.toPandas().head(20)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BP>70,skinthickness_group,Outcome_label,Outcome_groupIndex
0,6,148,72,35,0,33.6,0.627,50,1,1,other,One,1.0
1,1,85,66,29,0,26.6,0.351,31,0,0,other,Zero,0.0
2,8,183,64,0,0,23.3,0.672,32,1,0,SK1,One,1.0
3,1,89,66,23,94,28.1,0.167,21,0,0,other,Zero,0.0
4,0,137,40,35,168,43.1,2.288,33,1,0,other,One,1.0
5,5,116,74,0,0,25.6,0.201,30,0,1,SK1,Zero,0.0
6,3,78,50,32,88,31.0,0.248,26,1,0,other,One,1.0
7,10,115,0,0,0,35.3,0.134,29,0,0,SK1,Zero,0.0
8,2,197,70,45,543,30.5,0.158,53,1,0,other,One,1.0
9,8,125,96,0,0,0.0,0.232,54,1,1,SK1,One,1.0


### 6. One hot encoder

In [27]:
encoder = OneHotEncoder()\
         .setInputCols (["skinthickness_groupIndex"])\
         .setOutputCols (["SkinThickness_encoded"])

encoder_model=encoder.fit(skinthickness_groupIndex_df)
encoder_df=encoder_model.transform(skinthickness_groupIndex_df)

encoder_df.toPandas().head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BP>70,skinthickness_group,Outcome_label,skinthickness_groupIndex,SkinThickness_encoded
0,6,148,72,35,0,33.6,0.627,50,1,1,other,One,0.0,"(1.0, 0.0, 0.0)"
1,1,85,66,29,0,26.6,0.351,31,0,0,other,Zero,0.0,"(1.0, 0.0, 0.0)"
2,8,183,64,0,0,23.3,0.672,32,1,0,SK1,One,1.0,"(0.0, 1.0, 0.0)"
3,1,89,66,23,94,28.1,0.167,21,0,0,other,Zero,0.0,"(1.0, 0.0, 0.0)"
4,0,137,40,35,168,43.1,2.288,33,1,0,other,One,0.0,"(1.0, 0.0, 0.0)"


In [28]:
encoder = OneHotEncoder()\
         .setInputCols (["Outcome_groupIndex"])\
         .setOutputCols (["Outcome_encoded"])

encoder_model=encoder.fit(Outcome_groupIndex_df)
encoder_df=encoder_model.transform(Outcome_groupIndex_df)

encoder_df.toPandas().head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,BP>70,skinthickness_group,Outcome_label,Outcome_groupIndex,Outcome_encoded
0,6,148,72,35,0,33.6,0.627,50,1,1,other,One,1.0,(0.0)
1,1,85,66,29,0,26.6,0.351,31,0,0,other,Zero,0.0,(1.0)
2,8,183,64,0,0,23.3,0.672,32,1,0,SK1,One,1.0,(0.0)
3,1,89,66,23,94,28.1,0.167,21,0,0,other,Zero,0.0,(1.0)
4,0,137,40,35,168,43.1,2.288,33,1,0,other,One,1.0,(0.0)


### 7. Label encoder

In [29]:
label_indexer = StringIndexer()\
         .setInputCol ("skinthickness_group")\
         .setOutputCol ("label")

label_indexer_model=label_indexer.fit(encoder_df)
label_indexer_df=label_indexer_model.transform(encoder_df)

label_indexer_df.select("skinthickness_group","label").toPandas().head()

,skinthickness_group,label
0,other,0.0
1,other,0.0
2,SK1,1.0
3,other,0.0
4,other,0.0


In [30]:
label_indexer = StringIndexer()\
         .setInputCol ("Outcome_label")\
         .setOutputCol ("label")

label_indexer_model=label_indexer.fit(encoder_df)
label_indexer_df=label_indexer_model.transform(encoder_df)

label_indexer_df.select("Outcome_label","label").toPandas().head()

,Outcome_label,label
0,One,1.0
1,Zero,0.0
2,One,1.0
3,Zero,0.0
4,One,1.0


### 8. standard scaler

In [31]:
assembler = VectorAssembler()\
         .setInputCols (["Age","Pregnancies","Glucose",
                         "BloodPressure","SkinThickness",\
                         "Insulin","BMI","DiabetesPedigreeFunction"])\
         .setOutputCol ("vectorized_features")
        
# In case of missing you can skip the invalid ones
assembler_df=assembler.setHandleInvalid("skip").transform(encoder_df)

In [32]:
scaler = StandardScaler()\
         .setInputCol ("vectorized_features")\
         .setOutputCol ("features")
        
scaler_model=scaler.fit(assembler_df)
scaler_df=scaler_model.transform(assembler_df)
pd.set_option('display.max_colwidth', 40)
scaler_df.select("vectorized_features","features").toPandas().head(5)

,vectorized_features,features
0,"[50.0, 6.0, 148.0, 72.0, 35.0, 0.0, ...","[4.251616970894646, 1.78063837321943..."
1,"[31.0, 1.0, 85.0, 66.0, 29.0, 0.0, 2...","[2.6360025219546803, 0.2967730622032..."
2,"[32.0, 8.0, 183.0, 64.0, 0.0, 0.0, 2...","[2.721034861372573, 2.37418449762590..."
3,"[21.0, 1.0, 89.0, 66.0, 23.0, 94.0, ...","[1.785679127775751, 0.29677306220323..."
4,"[33.0, 0.0, 137.0, 40.0, 35.0, 168.0...","[2.806067200790466, 0.0, 4.284916523..."


### 9. Vector assembler

In [33]:
import pandas as pd
pd.set_option('display.max_colwidth', 80)
pd.set_option("display.max_columns", 12)

In [34]:
assembler = VectorAssembler()\
         .setInputCols (["Age","Pregnancies","Glucose",
                         "BloodPressure","SkinThickness",\
                         "Insulin","BMI","DiabetesPedigreeFunction"])\
         .setOutputCol ("vectorized_features")
        
# In case of missing you can skip the invalid ones
assembler_df=assembler.setHandleInvalid("skip").transform(encoder_df)
assembler_df.toPandas().head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,...,BP>70,skinthickness_group,Outcome_label,Outcome_groupIndex,Outcome_encoded,vectorized_features
0,6,148,72,35,0,33.6,...,1,other,One,1.0,(0.0),"[50.0, 6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627]"
1,1,85,66,29,0,26.6,...,0,other,Zero,0.0,(1.0),"[31.0, 1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351]"
2,8,183,64,0,0,23.3,...,0,SK1,One,1.0,(0.0),"[32.0, 8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672]"
3,1,89,66,23,94,28.1,...,0,other,Zero,0.0,(1.0),"[21.0, 1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167]"
4,0,137,40,35,168,43.1,...,0,other,One,1.0,(0.0),"[33.0, 0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288]"


### 10. Convert df to pandas and Perform the pearson coefficient - Target variable (Outcome) vs all input variables individually

In [35]:
pandas_df = assembler_df.toPandas()

In [36]:
int_columns = pandas_df.select_dtypes(include=["int"])

In [37]:
float_columns = pandas_df.select_dtypes(include=["float"])

In [38]:
numeric_columns = int_columns + float_columns

In [39]:
outcome_correlations = {}

for column in numeric_columns.columns:
    if column != "Outcome":
        correlation = pandas_df["Outcome"].corr(pandas_df[column])
        outcome_correlations[column] = correlation


outcome_correlations

{'Age': 0.23835598302719782,
 'BMI': 0.29269466264444544,
 'BP>70': 0.11946333987891701,
 'BloodPressure': 0.06506835955033281,
 'DiabetesPedigreeFunction': 0.17384406565296,
 'Glucose': 0.46658139830687406,
 'Insulin': 0.13054795488404788,
 'Outcome_groupIndex': 0.9999999999999999,
 'Pregnancies': 0.22189815303398638,
 'SkinThickness': 0.07475223191831955}